### Transformations using API data
I have discovered a new resource for my project that takes the US.gov data and aggregates by year.
This data is provided by the St. Louid FED through their FED program.
My hope through this exercise is to dump the 'total cellular subscription data' into a data frame.
Ethical considerations is to use a private API key to access the FRED database

### Transformation 1- rename columns

In [1]:
import requests
import pandas as pd

# hard coding API key and FRED database series ID
api_key = "0595639b15d0345a4fe7a22d0043342c"
series_id = "ITCELSETSP2USA"

# making the get data call
url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"
response = requests.get(url)

data = response.json()

# For now I think I only want the oberservations data. I may need more later.
observations = data["observations"]

data_list = [(obs["date"], obs["value"]) for obs in observations]

FREDcellData = pd.DataFrame(data_list, columns=["Date", "Number of Users"])

FREDcellData.head(500)


,Date,Number of Users
0,1960-01-01,0
1,1961-01-01,.
2,1962-01-01,.
3,1963-01-01,.
4,1964-01-01,.
...,...,...
57,2017-01-01,103.1297891
58,2018-01-01,104.847944
59,2019-01-01,106.4140195
60,2020-01-01,104.9353748


In [4]:
FREDcellData.to_csv('purchases.csv', index=False)

In [6]:
# Connect to the database
conn = sqlite3.connect('Projectdata.db')

# Write the DataFrame to a SQLite3 table
FREDcellData.to_sql(name='purchases', con=conn, if_exists='replace', index=False)

# Close the connection
conn.close()

### Transformation 2 Remove Rows with Bad Data

In [166]:
# I would like to remove number of user rows with 0 or blank
# going to try this in one shot

FREDcellDataCLEANED = FREDcellData[(FREDcellData['Number of Users'].notna()) & (FREDcellData['Number of Users'] != 0)]

FREDcellDataCLEANED.head(500)
# that did not work. possiblly because the cells are not actually blank. a '.' appears in the preview

,Date,Number of Users
0,1960-01-01,0
1,1961-01-01,.
2,1962-01-01,.
3,1963-01-01,.
4,1964-01-01,.
...,...,...
57,2017-01-01,103.1297891
58,2018-01-01,104.847944
59,2019-01-01,106.4140195
60,2020-01-01,104.9353748


In [167]:
# I think it's because I didn't wirte the script to remove the rows with a '.' or '0'
# let's try this

FREDcellDataCLEANED = FREDcellData[~FREDcellData['Number of Users'].isin(['.', '0'])]

FREDcellDataCLEANED.head(10)

# IT WORKED!
# the number of users data is in decimal, which matches the format of my website data. I will need to remember that it's in millions for the visualization


,Date,Number of Users
24,1984-01-01,0.039352779
25,1985-01-01,0.14468149
26,1986-01-01,0.28706876
27,1987-01-01,0.513170208
28,1988-01-01,0.854125086
29,1989-01-01,1.432490449
30,1990-01-01,2.129545117
31,1991-01-01,3.004111275
32,1992-01-01,4.323596882
33,1993-01-01,6.186519908


### Transformation 3 Remove Rows Prior to 2017

In [168]:
# This one is tough. I only need rows with data from 2017 and forward to compare to my other website data. 

FREDcellDataCLEANED = FREDcellDataCLEANED[FREDcellDataCLEANED['Date'] >= '2017-01-01']

FREDcellDataCLEANED.head(10)
# IT WORKED! YAY

,Date,Number of Users
57,2017-01-01,103.1297891
58,2018-01-01,104.847944
59,2019-01-01,106.4140195
60,2020-01-01,104.9353748
61,2021-01-01,107.3055043


### Transformation 4 - aggregate by year

In [47]:
# Calling number of new computers purchased data to add to FREDcellDataCLEANED data set

# hard coding API key and FRED database series ID
api_key = "0595639b15d0345a4fe7a22d0043342c"
series_id2 = "A34SNO"

# making the get data call
url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id2}&api_key={api_key}&file_type=json"
response2 = requests.get(url)

data = response.json()

# For now I think I only want the oberservations data. I may need more later.
observations2 = data["observations"]

data_list2 = [(obs["date"], obs["value"]) for obs in observations2]

# I think I need to convert the 'New Computer Purchases' column to int before running group by sum

FREDcomputerPurchase = pd.DataFrame(data_list2, columns=["Date", "New Computer Purchases"])

FREDcomputerPurchase.head(500)

,Date,New Computer Purchases
0,1992-02-01,19538
1,1992-03-01,19720
2,1992-04-01,19017
3,1992-05-01,20433
4,1992-06-01,19551
...,...,...
366,2022-08-01,24519
367,2022-09-01,24239
368,2022-10-01,24188
369,2022-11-01,24315


In [157]:
# Removing rows older than 2017
# Noticed that the computer purchased data set is by month FRED, I need to combine all of the months into a year

FREDcomputerPurchase = FREDcomputerPurchase[FREDcomputerPurchase['Date'] >= '2017-01-01']

FREDcomputerPurchase['New Computer Purchases'] = FREDcomputerPurchase['New Computer Purchases'].astype(int)

FREDcomputerPurchase.head(12)

,Date,New Computer Purchases
299,2017-01-01,21297
300,2017-02-01,21660
301,2017-03-01,21015
302,2017-04-01,21311
303,2017-05-01,21260
304,2017-06-01,20951
305,2017-07-01,21871
306,2017-08-01,21755
307,2017-09-01,22538
308,2017-10-01,22284


In [182]:
# I am going to try a group by function from pandas. Not sure if this will work or not. 
import pandas as pd

# Kept getting error for the 'Date' column being index and not date/time. Going to try and convert
FREDcomputerPurchase['Date'] = pd.to_datetime(FREDcomputerPurchase['Date'])

FREDcomputerPurchaseYR = FREDcomputerPurchase.groupby(pd.Grouper(key='Date', freq='Y')).sum()

FREDcomputerPurchaseYR

,New Computer Purchases
Date,
2017-12-31,260553
2018-12-31,267756
2019-12-31,272660
2020-12-31,264325
2021-12-31,275244
2022-12-31,288546


In [191]:
# Making sure the data frames have the same amount of rows to concatenate 

FREDcomputerPurchaseStripped = FREDcomputerPurchaseYR.head(5)
FREDcomputerPurchaseStripped

,New Computer Purchases
Date,
2017-12-31,260553
2018-12-31,267756
2019-12-31,272660
2020-12-31,264325
2021-12-31,275244


### Transformation #5- Add data from FREDcomputerPurchaseStripped to FREDcellDataCLEANED

In [203]:
import numpy as np
result = pd.concat([FREDcomputerPurchaseStripped['New Computer Purchases'], FREDcellDataCLEANED], axis=1)
result

,New Computer Purchases,Date,Number of Users
2017-12-31 00:00:00,260553.0,NaN,NaN
2018-12-31 00:00:00,267756.0,NaN,NaN
2019-12-31 00:00:00,272660.0,NaN,NaN
2020-12-31 00:00:00,264325.0,NaN,NaN
2021-12-31 00:00:00,275244.0,NaN,NaN
57,NaN,2017-01-01,103.1297891
58,NaN,2018-01-01,104.847944
59,NaN,2019-01-01,106.4140195
60,NaN,2020-01-01,104.9353748
61,NaN,2021-01-01,107.3055043


In [204]:
result['Date'] = result['Date'].shift(-5, fill_value=np.nan)

# View the resulting DataFrame
print(result)

                     New Computer Purchases        Date Number of Users
2017-12-31 00:00:00                260553.0  2017-01-01             NaN
2018-12-31 00:00:00                267756.0  2018-01-01             NaN
2019-12-31 00:00:00                272660.0  2019-01-01             NaN
2020-12-31 00:00:00                264325.0  2020-01-01             NaN
2021-12-31 00:00:00                275244.0  2021-01-01             NaN
57                                      NaN         NaN     103.1297891
58                                      NaN         NaN      104.847944
59                                      NaN         NaN     106.4140195
60                                      NaN         NaN     104.9353748
61                                      NaN         NaN     107.3055043


In [205]:
# My transofrmation is sloppy but I have the number of cellphone users and new computer purchases in one variable. 
# Struggle was real. 
result['Number of Users'] = result['Number of Users'].shift(-5, fill_value=np.nan)

# View the resulting DataFrame
print(result)

                     New Computer Purchases        Date Number of Users
2017-12-31 00:00:00                260553.0  2017-01-01     103.1297891
2018-12-31 00:00:00                267756.0  2018-01-01      104.847944
2019-12-31 00:00:00                272660.0  2019-01-01     106.4140195
2020-12-31 00:00:00                264325.0  2020-01-01     104.9353748
2021-12-31 00:00:00                275244.0  2021-01-01     107.3055043
57                                      NaN         NaN             NaN
58                                      NaN         NaN             NaN
59                                      NaN         NaN             NaN
60                                      NaN         NaN             NaN
61                                      NaN         NaN             NaN


In [3]:
import sqlite3
import pandas as pd

# Define your data as a Pandas DataFrame
data = result.DataFrame({'New Computer Purchases': [1, 2, 3], 'Column2': ['A', 'B', 'C']})

# Connect to the database
conn = sqlite3.connect('computerpurchases.db')

# Write the DataFrame to a SQLite3 table
data.to_sql(name='purchases', con=conn, if_exists='replace', index=False)

# Close the connection
conn.close()

NameError: name 'result' is not defined